## Notebook to test the c find functionality in pynetdicom

In [1]:
from pydicom.dataset import Dataset
from pynetdicom import AE
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelFind

In [3]:
from pynetdicom import (
    AE, evt, build_role,
    PYNETDICOM_IMPLEMENTATION_UID,
    PYNETDICOM_IMPLEMENTATION_VERSION,
    StoragePresentationContexts
)
from pynetdicom.sop_class import (
    PatientRootQueryRetrieveInformationModelMove,
    CTImageStorage,
    ComputedRadiographyImageStorage
)

## Start the SCP

In [46]:
# Implement the handler for evt.EVT_C_STORE
def handle_store(event):
    """Handle a C-STORE request event."""
    ds = event.dataset
    context = event.context

    # Add the DICOM File Meta Information
    meta = Dataset()
    meta.MediaStorageSOPClassUID = ds.SOPClassUID
    meta.MediaStorageSOPInstanceUID = ds.SOPInstanceUID
    meta.ImplementationClassUID = PYNETDICOM_IMPLEMENTATION_UID
    meta.ImplementationVersionName = PYNETDICOM_IMPLEMENTATION_VERSION
    meta.TransferSyntaxUID = context.transfer_syntax

    # Add the file meta to the dataset
    ds.file_meta = meta

    # Set the transfer syntax attributes of the dataset
    ds.is_little_endian = context.transfer_syntax.is_little_endian
    ds.is_implicit_VR = context.transfer_syntax.is_implicit_VR

    # Save the dataset using the SOP Instance UID as the filename
    ds.save_as(ds.SOPInstanceUID, write_like_original=False)

    # Return a 'Success' status
    return 0x0000

handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE(ae_title = 'RBTEST')

# Add the requested presentation contexts (QR SCU)
ae.add_requested_context(PatientRootQueryRetrieveInformationModelMove)
# Add the requested presentation context (Storage SCP)
#ae.add_requested_context(CTImageStorage)
#ae.add_requested_contexts = StoragePresentationContexts
ae.supported_contexts = StoragePresentationContexts

scp = ae.start_server(('', 4284), block=False, evt_handlers=handlers)

In [47]:

# Initialise the Application Entity
#ae = AE(ae_title = "RBTEST")

# Add a requested presentation context
ae.add_requested_context(PatientRootQueryRetrieveInformationModelFind)

# Create our Identifier (query) dataset
ds = Dataset()
#ds.SOPClassesInStudy = ''
#ds.PatientID = 'M0003'
ds.PatientID = "*"
ds.AccessionNumber = "934531"
ds.StudyInstanceUID = '*'
ds.QueryRetrieveLevel = 'PATIENT'
ds.SeriesInstanceUID = "*"
#ds.ReferencedInstanceSequence = "*"

In [48]:
# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate('10.200.200.201', port=104)

if assoc.is_established:
    # Use the C-FIND service to send the identifier
    responses = assoc.send_c_find(ds, query_model = PatientRootQueryRetrieveInformationModelFind)
    for (status, identifier) in responses:
        if status:
            print('C-FIND query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then identifier is the C-FIND response
            if status.Status in (0xFF00, 0xFF01):
                ds = identifier
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

     # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

C-FIND query status: 0xff00
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0050) Accession Number                    SH: '934531'
(0008, 0052) Query/Retrieve Level                CS: 'PATIENT'
(0010, 0020) Patient ID                          LO: '4482356'
(0020, 000d) Study Instance UID                  UI: 1.2.840.113745.101000.1186002.40721.8111.13846623
(0020, 000e) Series Instance UID                 UI: 1.2.276.0.7230010.3.1.3.0.32423.1424660650.923422
C-FIND query status: 0x0000


## Implement C - move with SCP

In [49]:
from pydicom.dataset import Dataset

from pynetdicom import AE
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelMove

In [50]:
ds.QueryRetrieveLevel = 'PATIENT'
delattr(ds, 'AccessionNumber')
orig_ds = ds

In [51]:
ds

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0052) Query/Retrieve Level                CS: 'PATIENT'
(0010, 0020) Patient ID                          LO: '4482356'
(0020, 000d) Study Instance UID                  UI: 1.2.840.113745.101000.1186002.40721.8111.13846623
(0020, 000e) Series Instance UID                 UI: 1.2.276.0.7230010.3.1.3.0.32423.1424660650.923422

In [52]:
# Create our Identifier (query) dataset
# We need to supply a Unique Key Attribute for each level above the
#   Query/Retrieve level
ds = orig_ds

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate('10.200.200.201', 104)

if assoc.is_established:
    # Use the C-GET service to send the identifier
    responses = assoc.send_c_move(ds, 'RBTEST', PatientRootQueryRetrieveInformationModelMove)

    for (status, identifier) in responses:
        if status:
            print('C-MOVE query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then `identifier` is the C-GET response
            if status.Status in (0xFF00, 0xFF01):
                ct_ds = identifier
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')
    
#scp.shutdown()

C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0xff00
C-MOVE query status: 0x0000


In [45]:
scp.shutdown()

## Implement C-move

In [ ]:
from pydicom.dataset import Dataset

from pynetdicom import AE
from pynetdicom.sop_class import StudyRootQueryRetrieveInformationModelMove, PatientRootQueryRetrieveInformationModelMove

# Initialise the Application Entity
ae = AE(ae_title = "RBTEST")

# Add a requested presentation context
ae.add_requested_context(PatientRootQueryRetrieveInformationModelMove)

# Create out identifier (query) dataset
#ds = Dataset()
#ds.QueryRetrieveLevel = 'SERIES'
# Unique key for PATIENT level
#ds.PatientID = '1234567'
# Unique key for STUDY level
#ds.StudyInstanceUID = '1.2.3'
# Unique key for SERIES level
#ds.SeriesInstanceUID = '1.2.3.4'

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate('10.200.200.201', 4242)

if assoc.is_established:
    # Use the C-MOVE service to send the identifier
    responses = assoc.send_c_move(ds, b'RADIOBOTICS-PACS', PatientRootQueryRetrieveInformationModelMove)

    for (status, identifier) in responses:
        if status:
            print('C-MOVE query status: 0x{0:04x}'.format(status.Status))
            ds = identifier
            # If the status is 'Pending' then the identifier is the C-MOVE response
            if status.Status in (0xFF00, 0xFF01):
                
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

In [ ]:
ds

## Implement C-Get

In [ ]:
from pydicom.dataset import Dataset

from pynetdicom import (
    AE, evt, build_role,
    PYNETDICOM_IMPLEMENTATION_UID,
    PYNETDICOM_IMPLEMENTATION_VERSION,
    StoragePresentationContexts
)
from pynetdicom.sop_class import (
    PatientRootQueryRetrieveInformationModelGet,
    CTImageStorage,
    DigitalXRayImagePresentationStorage,
    ComputedRadiographyImageStorage
)

In [ ]:
# Create our Identifier (query) dataset
# We need to supply a Unique Key Attribute for each level above the
#   Query/Retrieve level


In [ ]:

ds.QueryRetrieveLevel = 'SERIES'
# Unique key for PATIENT level
#ds.PatientID = '1234567'
# Unique key for STUDY level
#ds.StudyInstanceUID = '1.2.3'
# Unique key for SERIES level
#ds.SeriesInstanceUID = '1.2.3.4'

In [ ]:
ds

In [ ]:
handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE()

# Add the requested presentation contexts (QR SCU)
ae.add_requested_context(PatientRootQueryRetrieveInformationModelGet)
# Add the requested presentation context (Storage SCP)
ae.add_requested_context(ComputedRadiographyImageStorage)

# Create an SCP/SCU Role Selection Negotiation item for CT Image Storage
role = build_role(ComputedRadiographyImageStorage, scp_role=True)


# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate('10.200.200.201', port=4242, ext_neg=[role], evt_handlers=handlers)

if assoc.is_established:
    print("ME")
    # Use the C-GET service to send the identifier
    responses = assoc.send_c_get(ds, PatientRootQueryRetrieveInformationModelGet)

    for (status, identifier) in responses:
        if status:
            print('C-GET query status: 0x{0:04x}'.format(status.Status))w

            # If the status is 'Pending' then `identifier` is the C-GET response
            if status.Status in (0xFF00, 0xFF01):
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

In [ ]:
# Implement the handler for evt.EVT_C_STORE
def handle_store(event):
    """Handle a C-STORE request event."""
    ds = event.dataset
    context = event.context

    # Add the DICOM File Meta Information
    meta = Dataset()
    meta.MediaStorageSOPClassUID = ds.SOPClassUID
    meta.MediaStorageSOPInstanceUID = ds.SOPInstanceUID
    meta.ImplementationClassUID = PYNETDICOM_IMPLEMENTATION_UID
    meta.ImplementationVersionName = PYNETDICOM_IMPLEMENTATION_VERSION
    meta.TransferSyntaxUID = context.transfer_syntax

    # Add the file meta to the dataset
    ds.file_meta = meta

    # Set the transfer syntax attributes of the dataset
    ds.is_little_endian = context.transfer_syntax.is_little_endian
    ds.is_implicit_VR = context.transfer_syntax.is_implicit_VR

    # Save the dataset using the SOP Instance UID as the filename
    ds.save_as(ds.SOPInstanceUID, write_like_original=False)

    # Return a 'Success' status
    return 0x0000

In [ ]:

    # Add the DICOM File Meta Information
    meta = Dataset()
    meta.MediaStorageSOPClassUID = ds.SOPClassUID
    meta.MediaStorageSOPInstanceUID = ds.SOPInstanceUID
    meta.ImplementationClassUID = PYNETDICOM_IMPLEMENTATION_UID
    meta.ImplementationVersionName = PYNETDICOM_IMPLEMENTATION_VERSION
    meta.TransferSyntaxUID = context.transfer_syntax

    # Add the file meta to the dataset
    ds.file_meta = meta

    # Set the transfer syntax attributes of the dataset
    ds.is_little_endian = context.transfer_syntax.is_little_endian
    ds.is_implicit_VR = context.transfer_syntax.is_implicit_VR

    # Save the dataset using the SOP Instance UID as the filename
    ds.save_as(ds.SOPInstanceUID, write_like_original=False)